In [1]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

ModuleNotFoundError: No module named 'torch'

In [ ]:
env = gym.make('CartPole-v1')

for episode in range(20):
    
    env.reset() 
    is_done = False

    while not is_done:
        action = env.action_space.sample()
        new_state, reward, is_done, info = env.step(action)
        env.render()

In [ ]:
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4

n_actions = env.action_space.n
state = env.reset()
n_observations = len(state)

In [10]:
# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class DQN(nn.Module):
    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

policy_net = DQN(n_observations, n_actions).to(device)
optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)

policy_net

DQN(
  (layer1): Linear(in_features=4, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=128, bias=True)
  (layer3): Linear(in_features=128, out_features=2, bias=True)
)

In [17]:
def select_action(state):
    sample = np.random.random()
    eps_threshold = 0.2
    if sample > eps_threshold:
        with torch.no_grad():
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)

state = env.reset()
state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
select_action(state)

tensor([[0]], device='cuda:0')

In [23]:
done = False

action = select_action(state)
observation, reward, done, info = env.step(action.item())
reward = torch.tensor([reward], device=device)

In [25]:
done

False

In [ ]:
state_action_values = policy_net(state).gather(1, action_batch)

    with torch.no_grad():
        next_state_values = policy_net(observation).max(1)[0]
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    # torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

    next_state = None
else:
    next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)